# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/dsECZ/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298672202851                   -0.85    4.9         
  2   -8.300229358841       -2.81       -1.26    1.6    162ms
  3   -8.300440577648       -3.68       -1.87    2.4    247ms
  4   -8.300462203230       -4.67       -2.70    1.9    160ms
  5   -8.300464398553       -5.66       -3.22    3.0    212ms
  6   -8.300464518951       -6.92       -3.34   11.0    375ms
  7   -8.300464573284       -7.26       -3.48    2.1    196ms
  8   -8.300464607693       -7.46       -3.62    1.1    196ms
  9   -8.300464635830       -7.55       -3.84    1.5    171ms
 10   -8.300464639915       -8.39       -3.96    5.4    265ms
 11   -8.300464643644       -8.43       -4.29    1.9    215ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.66895820810                   -0.70    7.0         
  2   -16.67852332975       -2.02       -1.14    1.4    424ms
  3   -16.67919678424       -3.17       -1.86    3.6    532ms
  4   -16.67927291702       -4.12       -2.70    2.2    460ms
  5   -16.67928528203       -4.91       -3.07    5.1    610ms
  6   -16.67928619957       -6.04       -3.49    2.1    439ms
  7   -16.67928621759       -7.74       -3.95    2.2    449ms
  8   -16.67928622162       -8.40       -4.53    2.2    454ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32590069083                   -0.56    7.6         
  2   -33.33424276732       -2.08       -1.00    1.6    1.52s
  3   -33.33597932127       -2.76       -1.72    4.9    1.97s
  4   -33.33614431614       -3.78       -2.63    2.5    1.70s
  5   -33.33685278916       -3.15       -2.36    8.8    2.59s
  6   -33.33688372448       -4.51       -2.45    3.5    1.57s
  7   -33.33694328825       -4.23       -3.52    2.0    1.68s
  8   -33.33694362834       -6.47       -3.68    4.6    2.24s
  9   -33.33694376995       -6.85       -4.15    1.5    1.47s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298560181259                   -0.85    5.1         
  2   -8.300275382524       -2.77       -1.58    1.0    131ms
  3   -8.300360705038       -4.07       -2.25    2.6    175ms
  4   -8.300323443024   +   -4.43       -2.18    3.5    213ms
  5   -8.300464114820       -3.85       -3.56    1.0    145ms
  6   -8.300464573979       -6.34       -3.81    6.9    362ms
  7   -8.300464637069       -7.20       -4.19    1.0    150ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32571509517                   -0.56    7.0         
  2   -33.31544024112   +   -1.99       -1.27    1.0    1.67s
  3   -15.79264334495   +    1.24       -0.44    5.9    2.59s
  4   -33.31192920759        1.24       -1.76    5.8    2.59s
  5   -33.23879658087   +   -1.14       -1.32    4.4    2.17s
  6   -32.98769688156   +   -0.60       -1.29    4.2    2.07s
  7   -33.30976900438       -0.49       -1.83    4.5    2.00s
  8   -33.33555324218       -1.59       -2.18    2.8    1.58s
  9   -33.33561794332       -4.19       -2.39    2.5    1.70s
 10   -33.33684119808       -2.91       -2.81    2.1    1.51s
 11   -33.33692117398       -4.10       -3.16    3.2    1.66s
 12   -33.33693806555       -4.77      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.